In [1]:
import pandas as pd
import json
from sodapy import Socrata
import os
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
from config2 import app_token
from geojson import Feature, FeatureCollection, Point

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", none)

# Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cityofchicago.org",
#                  "du5ugsgl0w1g4ql37x78pjbc5",
#                  username= "dr.harmony",
#                  password= "F3ar&Loathing")

# # Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token)
#                  username= username,
#                  password= password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85ca-t3if", limit = 500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
results_df

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,dooring_i,work_zone_i,statements_taken_i,photos_taken_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,f0c23e9f6c05d2f7ecf972e13f92f7dd10e2cc840bf7fc...,2020-10-15T23:55:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1dbb28934feae9f440bd2e029ac9e8deeb57a3ecfa5c47...,2020-10-15T23:07:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5574ab6f165424ae96436c15732e669f6854a242ace3be...,2020-10-15T22:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6a9c582618b8248ca8c35ed08c2b039f26a1903e553876...,2020-10-15T22:00:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0fada7783a5c380816111da8babd0262b45d5f4d2e1d39...,2020-10-15T21:56:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447861,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447862,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447863,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447864,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [4]:
## CLEAN UP CRASHES DF ** DELETE NaN values in beats, lat/long, remove lat/long with 0 value ** this is key for the MERGE later
results_nadrop_df = results_df.dropna(subset = ['beat_of_occurrence', 'latitude', 'longitude'])
# results_df = results_df.dropna(subset = ['latitude'])
# results_df = results_df.dropna(subset = ['longitude'])
# results_nazero_df = results_nadrop_df[results_nadrop_df.latitude != 0]
# results_nazero_df = results_nazero_df[results_nazero_df.longitude != 0]
results_clean = results_nadrop_df.drop_duplicates(subset='crash_record_id', keep="first")
results_clean

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,dooring_i,work_zone_i,statements_taken_i,photos_taken_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,f0c23e9f6c05d2f7ecf972e13f92f7dd10e2cc840bf7fc...,2020-10-15T23:55:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1dbb28934feae9f440bd2e029ac9e8deeb57a3ecfa5c47...,2020-10-15T23:07:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5574ab6f165424ae96436c15732e669f6854a242ace3be...,2020-10-15T22:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6a9c582618b8248ca8c35ed08c2b039f26a1903e553876...,2020-10-15T22:00:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0fada7783a5c380816111da8babd0262b45d5f4d2e1d39...,2020-10-15T21:56:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447861,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447862,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447863,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447864,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [5]:
## DROP ZEROS FROM LAT IF PRESENT
results_drop_latlong_zeros = results_clean[ (results_clean['latitude'] == 0) & (results_clean['longitude'] == '0')].index 
  
# drop these given row 
# indexes from dataFrame 
results_clean.drop(results_drop_latlong_zeros, inplace = True) 
  
results_clean

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,dooring_i,work_zone_i,statements_taken_i,photos_taken_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,f0c23e9f6c05d2f7ecf972e13f92f7dd10e2cc840bf7fc...,2020-10-15T23:55:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1dbb28934feae9f440bd2e029ac9e8deeb57a3ecfa5c47...,2020-10-15T23:07:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5574ab6f165424ae96436c15732e669f6854a242ace3be...,2020-10-15T22:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6a9c582618b8248ca8c35ed08c2b039f26a1903e553876...,2020-10-15T22:00:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0fada7783a5c380816111da8babd0262b45d5f4d2e1d39...,2020-10-15T21:56:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447861,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447862,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447863,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447864,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [6]:
results_clean.reset_index(drop=True)

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,dooring_i,work_zone_i,statements_taken_i,photos_taken_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,f0c23e9f6c05d2f7ecf972e13f92f7dd10e2cc840bf7fc...,2020-10-15T23:55:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1dbb28934feae9f440bd2e029ac9e8deeb57a3ecfa5c47...,2020-10-15T23:07:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5574ab6f165424ae96436c15732e669f6854a242ace3be...,2020-10-15T22:48:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6a9c582618b8248ca8c35ed08c2b039f26a1903e553876...,2020-10-15T22:00:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0fada7783a5c380816111da8babd0262b45d5f4d2e1d39...,2020-10-15T21:56:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445432,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
445433,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
445434,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
445435,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [7]:
## Change 'crash_date' to Datetime data type
results_clean['crash_date'] = pd.to_datetime(results_clean['crash_date'])
results_clean.dtypes

crash_record_id                          object
crash_date                       datetime64[ns]
posted_speed_limit                       object
traffic_control_device                   object
device_condition                         object
weather_condition                        object
lighting_condition                       object
first_crash_type                         object
trafficway_type                          object
alignment                                object
roadway_surface_cond                     object
road_defect                              object
report_type                              object
crash_type                               object
hit_and_run_i                            object
damage                                   object
date_police_notified                     object
prim_contributory_cause                  object
sec_contributory_cause                   object
street_no                                object
street_direction                        

In [8]:
## FILTER OUT A YEAR (2019)
# results_df2019 = results_clean[results_clean['crash_date'].dt.year == 2019].reset_index(drop=True)

## FILTER OUT 2nd half of (2019)
#greater than the start date and smaller than the end date
mask = (results_clean['crash_date'] > "2019-07-30") & (results_clean['crash_date'] <= "2019-12-31")

results_df2019 = results_clean.loc[mask]
results_df2019

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,dooring_i,work_zone_i,statements_taken_i,photos_taken_i,work_zone_type,workers_present_i,rd_no,lane_cnt
73580,3dc6e111880ba404815659e581e50afabf1b6d94d3f8d7...,2019-12-31 00:00:00,5,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,PARKING LOT,STRAIGHT AND LEVEL,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,JD102299,NaN
73581,26700d215cb8c28d1dc6b589bb1329166985fd1b3a3a9a...,2019-12-30 23:43:00,35,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,Y,NaN,NaN,CONSTRUCTION,NaN,JC566017,NaN
73582,e50a88f02d4179242fe749f527cfd9f13e6ed3f594e337...,2019-12-30 23:30:00,20,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC566011,NaN
73583,25e6e00fa4528a4aeb9922c47a7ed15c5e78d13b3aeba0...,2019-12-30 23:15:00,30,UNKNOWN,UNKNOWN,SNOW,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,JC566028,NaN
73584,ad80bbbb1242b322a3c6097abe3823ed615b463873954b...,2019-12-30 23:13:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,BLOWING SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC566001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122664,5c4d4f9d8efc7a42a36707fd61e8e38723710defe04e66...,2019-07-30 00:30:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC372635,NaN
122665,4ab2a75da4b88e1516be288b00c104fd53620e128e6296...,2019-07-30 00:30:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC370457,NaN
122666,3de4bf3db775d9506a089ff32d3772ca5f627c5bd319ce...,2019-07-30 00:23:00,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC370443,NaN
122668,10cb8df2858e2c994aa46f298512d0d175b3db856fe456...,2019-07-30 00:15:00,30,STOP SIGN/FLASHER,FUNCTIONING IMPROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",HEAD ON,CENTER TURN LANE,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,JC370430,NaN


In [9]:
## DROP UNWANTED COLUMNS
crashes = results_df2019.drop(columns=['crash_record_id', 'location',
                                       'injuries_unknown', 'injuries_incapacitating', 'injuries_no_indication', 'injuries_reported_not_evident',
                                       'injuries_non_incapacitating', 'private_property_i', 'crash_date_est_i', 'statements_taken_i', 'photos_taken_i',
                                       'work_zone_i', 'work_zone_type', 'workers_present_i', 'dooring_i'])

In [10]:
crashes.rename(columns={"beat_of_occurrence": "Beat"}, inplace = True)

In [11]:
crashes.reset_index(drop=True)

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,...,injuries_total,injuries_fatal,crash_hour,crash_day_of_week,crash_month,latitude,longitude,intersection_related_i,rd_no,lane_cnt
0,2019-12-31 00:00:00,5,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,PARKING LOT,STRAIGHT AND LEVEL,UNKNOWN,...,0,0,0,3,12,41.86585735,-87.626935006,NaN,JD102299,NaN
1,2019-12-30 23:43:00,35,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,0,0,23,2,12,41.692559763,-87.621193695,NaN,JC566017,NaN
2,2019-12-30 23:30:00,20,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,0,0,23,2,12,41.968780354,-87.718119321,NaN,JC566011,NaN
3,2019-12-30 23:15:00,30,UNKNOWN,UNKNOWN,SNOW,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,1,0,23,2,12,41.756855835,-87.592814173,NaN,JC566028,NaN
4,2019-12-30 23:13:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,BLOWING SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,1,0,23,2,12,41.824566228,-87.694807376,Y,JC566001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48649,2019-07-30 00:30:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,...,0,0,0,3,7,41.920336172,-87.794994302,NaN,JC372635,NaN
48650,2019-07-30 00:30:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,0,0,0,3,7,41.84471264,-87.65192047,NaN,JC370457,NaN
48651,2019-07-30 00:23:00,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,2,0,0,3,7,41.978568312,-87.787966866,Y,JC370443,NaN
48652,2019-07-30 00:15:00,30,STOP SIGN/FLASHER,FUNCTIONING IMPROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",HEAD ON,CENTER TURN LANE,STRAIGHT AND LEVEL,DRY,...,0,0,0,3,7,41.791955376,-87.703659302,Y,JC370430,NaN


In [12]:
# CHANGE 'Beat' Column to int64. Just incase of merge
crashes['Beat'] = pd.to_numeric(crashes['Beat'])
crashes.dtypes

crash_date                 datetime64[ns]
posted_speed_limit                 object
traffic_control_device             object
device_condition                   object
weather_condition                  object
lighting_condition                 object
first_crash_type                   object
trafficway_type                    object
alignment                          object
roadway_surface_cond               object
road_defect                        object
report_type                        object
crash_type                         object
hit_and_run_i                      object
damage                             object
date_police_notified               object
prim_contributory_cause            object
sec_contributory_cause             object
street_no                          object
street_direction                   object
street_name                        object
Beat                                int64
num_units                          object
most_severe_injury                

In [13]:
## FOR GEOJSON EXPORT  ****
forgeojson = crashes.drop(columns=['posted_speed_limit', 'traffic_control_device',
       'device_condition', 'lighting_condition',
       'alignment', 'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'damage', 'date_police_notified', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'num_units', 'most_severe_injury',
       'injuries_total', 'crash_hour', 'crash_month','hit_and_run_i',
       'intersection_related_i', 'rd_no', 'lane_cnt'])
forgeojson

,crash_date,weather_condition,first_crash_type,trafficway_type,Beat,injuries_fatal,crash_day_of_week,latitude,longitude
73580,2019-12-31 00:00:00,UNKNOWN,PARKED MOTOR VEHICLE,PARKING LOT,131,0,3,41.86585735,-87.626935006
73581,2019-12-30 23:43:00,UNKNOWN,FIXED OBJECT,NOT DIVIDED,531,0,2,41.692559763,-87.621193695
73582,2019-12-30 23:30:00,SNOW,ANGLE,NOT DIVIDED,1712,0,2,41.968780354,-87.718119321
73583,2019-12-30 23:15:00,SNOW,REAR END,NOT DIVIDED,411,0,2,41.756855835,-87.592814173
73584,2019-12-30 23:13:00,BLOWING SNOW,ANGLE,FOUR WAY,911,0,2,41.824566228,-87.694807376
...,...,...,...,...,...,...,...,...,...
122664,2019-07-30 00:30:00,CLEAR,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),2512,0,3,41.920336172,-87.794994302
122665,2019-07-30 00:30:00,CLEAR,SIDESWIPE SAME DIRECTION,NOT DIVIDED,913,0,3,41.84471264,-87.65192047
122666,2019-07-30 00:23:00,CLEAR,ANGLE,NOT DIVIDED,1613,0,3,41.978568312,-87.787966866
122668,2019-07-30 00:15:00,CLEAR,HEAD ON,CENTER TURN LANE,822,0,3,41.791955376,-87.703659302


In [17]:
## REPLACE NaN values with 0 in injuries_fatal column
forgeojson['injuries_fatal'].fillna(value = 0,  
          inplace = True) 
forgeojson

,crash_date,weather_condition,first_crash_type,trafficway_type,Beat,injuries_fatal,crash_day_of_week,latitude,longitude
73580,2019-12-31 00:00:00,UNKNOWN,PARKED MOTOR VEHICLE,PARKING LOT,131,0,3,41.86585735,-87.626935006
73581,2019-12-30 23:43:00,UNKNOWN,FIXED OBJECT,NOT DIVIDED,531,0,2,41.692559763,-87.621193695
73582,2019-12-30 23:30:00,SNOW,ANGLE,NOT DIVIDED,1712,0,2,41.968780354,-87.718119321
73583,2019-12-30 23:15:00,SNOW,REAR END,NOT DIVIDED,411,0,2,41.756855835,-87.592814173
73584,2019-12-30 23:13:00,BLOWING SNOW,ANGLE,FOUR WAY,911,0,2,41.824566228,-87.694807376
...,...,...,...,...,...,...,...,...,...
122664,2019-07-30 00:30:00,CLEAR,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),2512,0,3,41.920336172,-87.794994302
122665,2019-07-30 00:30:00,CLEAR,SIDESWIPE SAME DIRECTION,NOT DIVIDED,913,0,3,41.84471264,-87.65192047
122666,2019-07-30 00:23:00,CLEAR,ANGLE,NOT DIVIDED,1613,0,3,41.978568312,-87.787966866
122668,2019-07-30 00:15:00,CLEAR,HEAD ON,CENTER TURN LANE,822,0,3,41.791955376,-87.703659302


In [18]:
forgeojsonsmlr = forgeojson.drop(columns=['crash_day_of_week', 'Beat', 'trafficway_type'])
forgeojsonsmlr.reset_index(drop=True)
forgeojsonsmlr

,crash_date,weather_condition,first_crash_type,injuries_fatal,latitude,longitude
73580,2019-12-31 00:00:00,UNKNOWN,PARKED MOTOR VEHICLE,0,41.86585735,-87.626935006
73581,2019-12-30 23:43:00,UNKNOWN,FIXED OBJECT,0,41.692559763,-87.621193695
73582,2019-12-30 23:30:00,SNOW,ANGLE,0,41.968780354,-87.718119321
73583,2019-12-30 23:15:00,SNOW,REAR END,0,41.756855835,-87.592814173
73584,2019-12-30 23:13:00,BLOWING SNOW,ANGLE,0,41.824566228,-87.694807376
...,...,...,...,...,...,...
122664,2019-07-30 00:30:00,CLEAR,REAR END,0,41.920336172,-87.794994302
122665,2019-07-30 00:30:00,CLEAR,SIDESWIPE SAME DIRECTION,0,41.84471264,-87.65192047
122666,2019-07-30 00:23:00,CLEAR,ANGLE,0,41.978568312,-87.787966866
122668,2019-07-30 00:15:00,CLEAR,HEAD ON,0,41.791955376,-87.703659302


In [19]:
# ## EXPORT DATA PULL FOR BU
# crashes.to_csv('accident_data_BU.txt', sep='\t', index=False)

In [20]:
## CREATING CRASHES AS GeoDataFrame
gdf_crashes2019 = gpd.GeoDataFrame(forgeojsonsmlr, geometry=gpd.points_from_xy(forgeojsonsmlr.longitude, forgeojsonsmlr.latitude))

In [21]:
## EXPORT CRASHES AS JSON FILE
gdf_crashes2019.to_file("crashes_in_2019.geojson", driver='GeoJSON')

In [ ]:
## FOR EXPORT TO NEIGHBORHOOD PARSER SCRIPT ****
forparse = crashes.drop(columns=['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'damage', 'date_police_notified', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'Beat', 'num_units', 'most_severe_injury',
       'injuries_total', 'injuries_fatal', 'crash_hour', 'crash_day_of_week',
       'crash_month','hit_and_run_i',
       'intersection_related_i', 'rd_no', 'lane_cnt'])
forparse.shape

In [ ]:
# ## EXPORTING LIST OF LAT LONG

# forparse.to_csv('forparse.txt', sep='\t', index=False)

In [ ]:
## READ PARSED LAT LONG WITH CHICAGO NEIGHBORHOOD
neighbor_lat_lon = pd.read_csv("static/data/parsed_community_with_polygon.csv")
neighbor_lat_lon

In [ ]:
## Split Neighborhood Name to 1 name after ','
neighbor_latlon_comname = neighbor_lat_lon.rename(columns={"Polygon Neighborhood": "communityArea"})
neighbor_latlon_comname

In [ ]:
neighbor_latlon_comname['communityArea'] = neighbor_latlon_comname['communityArea'].str.split(',').str[0]
neighbor_latlon_comname

In [ ]:
neighbor_latlon_comname.communityArea.unique()

In [ ]:
neighbor_latlon_comname["communityArea"].replace({"MIDWAY AIRPORT": "GARFIELD RIDGE", "GARFIELD PARK": "WEST & EAST GARFIELD PARK",
                                                 "RUSH & DIVISION": "NEAR NORTH SIDE"}, inplace=True)

In [ ]:
## READ Chicago Area geoJSON file for merge of Just Community Areas
chicagoAreas_df = gpd.read_file("static/data/chicago-community-areas.geojson")

In [ ]:
chicagoAreas_df.community.unique()


In [ ]:
## Remove all columns but the ones needed.
chicagoAreas_df_clean = chicagoAreas_df.drop(columns=['area', 'shape_area', 'perimeter', 'area_num_1',
       'comarea_id', 'comarea', 'shape_len', 'geometry'])
chicagoAreas_df_clean

In [ ]:
## Make sure Chicago Area name is changed for merge and column rename.
chicagoAreas_df_clean['area_numbe'] = pd.to_numeric(chicagoAreas_df_clean['area_numbe'])
chicagoAreas_df_clean

In [ ]:
## Column name change
chicago_areas_namechange = chicagoAreas_df_clean.rename(columns={"area_numbe": "community_num"})
pd.set_option("display.max_rows", None, "display.max_columns", None)
chicago_areas_namechange

In [ ]:
chicagoAreas_namechange = chicago_areas_namechange.rename(columns={"community": "communityArea"})
chicagoAreas_namechange

In [ ]:
# def generate_id(s):
#     return abs(hash(s)) % (10 ** 10)

In [ ]:
# chicagoAreas_namechange['id'] = chicagoAreas_namechange['communityArea'].apply(generate_id)
# chicagoAreas_namechange

In [ ]:
chicagoAreas_namechange.dtypes

In [ ]:
## *** THE MERGE TO GET COMMUNITY AREA NAME AND NUMBER ***
# crash_neighborhood_merge = pd.merge(left=neighbor_latlon_comname, right=chicagoAreas_namechange, how='left', on=['communityArea'])
crash_neighborhood_merge = neighbor_latlon_comname.merge(chicagoAreas_namechange, how='left', on=['communityArea'])

In [ ]:
pd.reset_option('^display.', silent=True)
crash_neighborhood_merge

In [ ]:
## FILL NaN
latlon_ChiAreas = crash_neighborhood_merge.set_index("communityArea").community_num.fillna(chicagoAreas_namechange.set_index("communityArea").community_num).reset_index()
latlon_ChiAreas

In [ ]:
# crash_neighborhood_merge = pd.concat([crashes, neighborhoods_df], axis=1)
crash_neighborhood_merge_clean = crash_neighborhood_merge[['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat', 'num_units',
       'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt']]
crash_neighborhood_merge_clean

In [ ]:
crash_neighborhood_merge_clean.columns

In [ ]:
crash_neighborhood_merge_clean

In [ ]:
## DROPPING COLUMNS
crashes_in_areas = crash_neighborhood_merge_clean.drop(columns=['posted_speed_limit', 'traffic_control_device', 'crash_type', 'device_condition', 'weather_condition', 'lighting_condition', 'trafficway_type', 'alignment',
'roadway_surface_cond', 'road_defect', 'report_type','intersection_related_i', 'damage', 'date_police_notified',
'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
'street_direction', 'street_name', 'District', 'num_units', 'most_severe_injury', 'crash_hour',
'crash_day_of_week', 'crash_month', 'rd_no', 'lane_cnt', 'hit_and_run_i'])

In [ ]:
crashes_in_areas

In [ ]:
## DROP any row with NaN values ** THIS REMOVES SOME CRASHES BUT KEEPS HIGH QUALITY RECORDED **
df = crashes_in_areas.dropna()
df

In [ ]:
## rename Community Area and Beat columns names
df = df.rename(columns = {"Community Area": "communityArea", "Beat": "beat"})
df

In [ ]:
df.dtypes

In [ ]:
# ## READ Chicago Area geoJSON file for merge
# chicagoAreas_df = gpd.read_file("static/data/chicago-community-areas.geojson")

In [ ]:
chicagoAreas_df
print(chicagoAreas_df)

In [ ]:
## REMOVE ** "area", "perimeter", "area_num_1", "comarea_id", "comarea"   ** from chicagoAreas_df
chicagoAreas_df = chicagoAreas_df.drop(columns=["area", "perimeter", "area_num_1", "comarea_id", "comarea"])
chicagoAreas_df = chicagoAreas_df.rename(columns = {"area_numbe": "communityArea"})
chicagoAreas_df.dtypes

In [ ]:
chicagoAreas_df[["communityArea"]] = chicagoAreas_df[["communityArea"]].apply(pd.to_numeric)
chicagoAreas_df.dtypes

In [ ]:
## MERGE THE TWO FILES TO CREATE MASTER 2019 GEOJSON
chicagoCrashMerge = pd.merge(left=df2019, right=chicagoAreas_df, how='right', on=['communityArea'])
chicagoCrashMerge

In [ ]:
chicagoCrashMerge['crashes_per_area'] = chicagoCrashMerge.groupby(['communityArea'])['community'].transform('count')
chicagoCrashMerge

In [ ]:
# CREATING GEOJSON FILE FOR EXPORT

## USING THE GEOPANDAS
# gdf_crashes2019 = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
gdf_crashes2019 = gpd.GeoDataFrame(chicagoCrashMerge)

In [ ]:
print(gdf_crashes2019.head())

In [ ]:
ax = gdf_crashes2019.dropna().plot(column='crashes_per_area', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=3, legend =  
                                  True);
#add title to the map
ax.set_title('Number of Crashes Per Chicago Area', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))
ax.get_figure()

In [ ]:
#gdf_crashes2019.to_file("crashes_in_2019.geojson", driver='GeoJSON')